Getting elevation from lat/long and making a new csv file which contains elevation


Importing necessary packages for getting data and making it into the final dataset.

In [1]:
import pandas as pd
import numpy as np

If you have the full dataset (CSV file) with the elevation, you won't need to run the next 4 cells.

***
Function to get elevation

In [2]:
import requests

# script for returning elevation from lat, long, based on open elevation data
# which in turn is based on SRTM
def get_elevation(lat: float, long: float):
    """
    :param lat: latitude coordinates of the point
    :param long: longitude coordinates of the point
    :return: elevation in meters as numpy.int64
    """
    query = ('https://api.open-elevation.com/api/v1/lookup'
             f'?locations={lat},{long}')
    r = requests.get(query).json()  # json object, various ways you can extract value
    # one approach is to use pandas json functionality:
    elevation = pd.json_normalize(r, 'results')['elevation'].values[0]
    return elevation

Getting the latitude and longitude points from the CSV file

In [3]:
dfFullNoElevation = pd.read_csv(filepath_or_buffer=r"D:\Google Drive\Uni\Tilburg\Semester 6\Thesis\Data\FW_Veg_Rem_Combined.csv")
dfLatLong = dfFullNoElevation[["latitude", "longitude"]]

# dfLatLong


Script to get the elevation. Takes a while. When I ran it, it took almost 10hrs to complete.

In [17]:
# import time  # optional for timing the code

dfElevation = pd.DataFrame()

# tic = time.perf_counter()  # timing start

for index, row in dfLatLong.iterrows():
    elevation = get_elevation(row["latitude"], row["longitude"])
    if index % 100 == 0:
        # print(f"{index} done.")  # useful for dev purposes, allows you to see the progress
        pass  # in case the printing is commented out
    dfElevation = dfElevation.append([elevation], ignore_index=True)

# toc = time.perf_counter()  # timing end
dfElevation.columns = ["elevation"]
dfElevation.head(10)
# print(f"Done in {toc - tic:0.4f} seconds")  # print the time elapsed


KeyboardInterrupt: 

Now we will want to get the day of the week of the discovery of the fire. First we convert the disc_clean_date to a datetime object, then from it, we get the day of the week.

Finally, in order to reduce the complexity of the data, I change it into a boolean column - weekdays are set to True and weekends to False.

In [15]:
dfFullNoElevation = pd.read_csv(filepath_or_buffer=r"D:\Google Drive\Uni\Tilburg\Semester 6\Thesis\Data\FW_Veg_Rem_Combined.csv")
dfFullNoElevation["disc_date_datetime"] = pd.to_datetime(dfFullNoElevation["disc_clean_date"], format="%m%d%Y", infer_datetime_format=True)  # create a new column which contains the datetime objects
dfFullNoElevation["disc_dow"] = dfFullNoElevation["disc_date_datetime"].dt.weekday  # create a column which contains the day of week (dow)

dfFullNoElevation["weekday"] = dfFullNoElevation["disc_dow"].map({0: True, 1: True, 2: True, 3: True, 4: True, 5:False, 6:False})
dfFullNoElevation.drop(labels="disc_dow", axis= 1, inplace=True)

Getting the elevation and date data into the full dataset. Writing it into a CSV file. This is useful so you don't have to run the code above every time you need to generate the data.

In [16]:
dfFull = pd.concat([dfFullNoElevation, dfElevation], axis=1)
#dfFull.head(10)
dfFull.to_csv(path_or_buf=r"D:\Google Drive\Uni\Tilburg\Semester 6\Thesis\Data\fullWithElevation.csv")

***

In case you already have the dataset (CSV file) with the elevation you can just run this part. It will import directly from the CSV into a Pandas DataFrame and run some necessary transformations/selections to produce the final dataset.

In [20]:
dfFull = pd.read_csv(filepath_or_buffer=r"D:\Google Drive\Uni\Tilburg\Semester 6\Thesis\Data\fullWithElevation.csv")
# dfFull.head(10)

dfFull = dfFull[["fire_size_class", "latitude", "longitude", "discovery_month", "weekday", "Temp_pre_30", "Temp_pre_15", "Temp_pre_7", "Wind_pre_30", "Wind_pre_15", "Wind_pre_7", "Hum_pre_30", "Hum_pre_15", "Hum_pre_7", "Prec_pre_30", "Prec_pre_15", "Prec_pre_7", "Vegetation", "remoteness", "elevation"]]

First, we can see there's some missing weather data. It's marked with a -1 rather than an NA. Let's fix that and try to drop the NA's.

In [21]:
print("before", dfFull.shape)

dfFull.replace(-1.0, np.NaN, inplace=True)
dfFull.dropna(inplace=True)
print("after", dfFull.shape)

before (55367, 20)
after (41118, 20)


Unfortunately we end up losing about 20% of our data...

> Perhaps, since we'll be splitting up the dataset, we can limit ourselves to a using this smaller dataset only when dealing with the weather variables.

Next up, we'll want to sort out our weather data.

Right now, for each weather variable, we have data from 30 days before, 15 days before, and 7 days before. In order to reduce the number of features, I take the average of the three values, store it in a new column in the DataFrame, and drop the old columns.

In [22]:
dfFull["Temp"] = dfFull.loc[:, ["Temp_pre_30", "Temp_pre_15", "Temp_pre_7"]].mean(axis=1)  # get the temperature
dfFull.drop(labels=["Temp_pre_30", "Temp_pre_15", "Temp_pre_7"], axis=1, inplace=True)  # drop the old temps

dfFull["Wind"] = dfFull.loc[:, ["Wind_pre_30", "Wind_pre_15", "Wind_pre_7"]].mean(axis=1)  # get the wind
dfFull.drop(labels=["Wind_pre_30", "Wind_pre_15", "Wind_pre_7"], axis=1, inplace=True)  # drop the old winds

dfFull["Humidity"] = dfFull.loc[:, ["Hum_pre_30", "Hum_pre_15", "Hum_pre_7"]].mean(axis=1)  # get the humidity
dfFull.drop(labels=["Hum_pre_30", "Hum_pre_15", "Hum_pre_7"], axis=1, inplace=True)  # drop the old humidities

dfFull["Precipitation"] = dfFull.loc[:, ["Prec_pre_30", "Prec_pre_15", "Prec_pre_7"]].mean(axis=1)  # get the humidity
dfFull.drop(labels=["Prec_pre_30", "Prec_pre_15", "Prec_pre_7"], axis=1, inplace=True)  # drop the old humidities

print(dfFull.shape)

(41118, 12)


Upon further inspection, it's evident that there's still some missing data. For some fires, all 4 weather variables are 0. It would be normal to see a 0 for one or two of them (for example, if there was no rain at all), but it seems wrong that it could happen for all 4.

Let's drop the rows in which all 4 weather columns == 0.

In [23]:
dfTemp = dfFull.query("Temp == 0 & Wind == 0 & Humidity == 0 & Precipitation == 0")  # create a new DataFrame that has only the 0 values

dfFull = dfFull.loc[dfFull.index.difference(dfTemp.index), ]
print(dfFull.shape)


(38562, 12)


After some research, I have found that having a relative humidity of 0% is pretty much an impossibility ([quick Google source](https://www.chicagotribune.com/news/ct-xpm-2011-12-16-ct-wea-1216-asktom-20111216-story.html)). Even if it's possible for short periods of time, it wouldn't be sustained over 30 days.

So, I've decided to consider such rows wrong (measurement error or missing data, take your pick) and thus drop them.

In [24]:
dfTemp = dfFull.query("Humidity == 0")

dfFull = dfFull.loc[dfFull.index.difference(dfTemp.index), ]
print(dfFull.shape)

(37247, 12)


Change the vegetation column's values from numbers into the actual category it is indicating.

In [25]:
dfFull.loc[(dfFull.Vegetation == 0), "Vegetation"] = np.NaN
dfFull.dropna(inplace=True)

dfFull.loc[(dfFull.Vegetation == 1), "Vegetation"] = "Tropical Evergreen Broadleaf Forest"
dfFull.loc[(dfFull.Vegetation == 2), "Vegetation"] = "Tropical Deciduous Broadleaf Forest"
dfFull.loc[(dfFull.Vegetation == 3), "Vegetation"] = "Temperate Evergreen Broadleaf Forest"
dfFull.loc[(dfFull.Vegetation == 4), "Vegetation"] = "Temperate Evergreen Needleleaf Forest"
dfFull.loc[(dfFull.Vegetation == 5), "Vegetation"] = "Temperate Deciduous Broadleaf Forest"
dfFull.loc[(dfFull.Vegetation == 6), "Vegetation"] = "Boreal Evergreen Needleleaf Forest"
dfFull.loc[(dfFull.Vegetation == 7), "Vegetation"] = "Boreal Deciduous Needleleaf Forest"
dfFull.loc[(dfFull.Vegetation == 8), "Vegetation"] = "Savanna"
dfFull.loc[(dfFull.Vegetation == 9), "Vegetation"] = "Grassland"
dfFull.loc[(dfFull.Vegetation == 10), "Vegetation"] = "Grassland"
dfFull.loc[(dfFull.Vegetation == 11), "Vegetation"] = "Dense Shrubland"
dfFull.loc[(dfFull.Vegetation == 12), "Vegetation"] = "Open Shrubland"
dfFull.loc[(dfFull.Vegetation == 13), "Vegetation"] = "Tundra"
dfFull.loc[(dfFull.Vegetation == 14), "Vegetation"] = "Desert"
dfFull.loc[(dfFull.Vegetation == 15), "Vegetation"] = "Rock/Ice"
dfFull.loc[(dfFull.Vegetation == 16), "Vegetation"] = "Tropical Evergreen Broadleaf Forest"
dfFull.loc[(dfFull.Vegetation == 17), "Vegetation"] = "Tropical Deciduous Broadleaf Forest"
dfFull.loc[(dfFull.Vegetation == 18), "Vegetation"] = "Temperate Evergreen Broadleaf Forest"
dfFull.loc[(dfFull.Vegetation == 19), "Vegetation"] = "Temperate Evergreen Needleleaf Forest"
dfFull.loc[(dfFull.Vegetation == 20), "Vegetation"] = "Temperate Deciduous Broadleaf Forest"
dfFull.loc[(dfFull.Vegetation == 21), "Vegetation"] = "Boreal Evergreen Needleleaf Forest"
dfFull.loc[(dfFull.Vegetation == 22), "Vegetation"] = "Boreal Deciduous Needleleaf Forest"
dfFull.loc[(dfFull.Vegetation == 23), "Vegetation"] = "Water"
dfFull.loc[(dfFull.Vegetation == 24), "Vegetation"] = "Cropland"
dfFull.loc[(dfFull.Vegetation == 25), "Vegetation"] = "Cropland"
dfFull.loc[(dfFull.Vegetation == 26), "Vegetation"] = "Pastureland"
dfFull.loc[(dfFull.Vegetation == 27), "Vegetation"] = "Pastureland"
dfFull.loc[(dfFull.Vegetation == 28), "Vegetation"] = "Urban"


Overall, the dataset's size has been reduced quite a lot. This is not ideal, but in starting out with over 50k samples, the dataset should still be large enough (30k samples) to perform our training and testing procedures.

> Hopefully, when not using the weather features, I will be able to use the full dataset.

Save the final dataset into a CSV.

In [26]:
dfFull.reset_index(drop=True, inplace=True)  # reset the index into a nice and clean one, discarding the old one
dfFull.to_csv(path_or_buf=r"fullDataFinal.csv")

We're done!
